# 16. Coroutines

- `b = yield [a]` 와 코루틴의 네가지 상태
- `next(...)` 와 `@coroutine` 을 이용한 priming (코루틴 기동)
- `.send(...)`, `.throw(...)`, `.close(...)`를 이용한 flow control
- `StopIteration` 예외와 코루틴에서의 return 
- `yield from x` 와 "delegating generator", "subgenerator", "caller"

- coroutine을 이용한 camera 예제 



### 코루틴 기본 동작

- 코루틴을 사용하기 위해서는 next(...) 로 priming(기동) 해야 한다.
- b = yield a 는 a를 출력하고 호출자(.send(...))로 받은 값을 b에 할당한다. a는 optional이다.

In [2]:
def simple_coro(a):
    print('-> Started: a =', a)
    b = yield a
    print('-> Received: b =', b)
    c = yield a + b
    print('-> Received: c =', c)

In [3]:
mycoro = simple_coro(14)
from inspect import getgeneratorstate

In [4]:
getgeneratorstate(mycoro)

'GEN_CREATED'

In [5]:
next(mycoro)

-> Started: a = 14


14

In [6]:
getgeneratorstate(mycoro)

'GEN_SUSPENDED'

In [7]:
mycoro.send(28)

-> Received: b = 28


42

In [8]:
getgeneratorstate(mycoro)

'GEN_SUSPENDED'

In [9]:
mycoro.send(99)

-> Received: c = 99


StopIteration: 

In [10]:
getgeneratorstate(mycoro)

'GEN_CLOSED'

- 코루틴의 실행은 yield에서 중단된다
- 코루틴은 다음 네가지 상태를 가진다. `inspect.getgeneratorstate()` 함수를 이용해서 현재 상태를 알 수 있다. 
    - 'GEN_CREATED'
    - 'GEN_RUNNING'
    - 'GEN_SUSPENDED'
    - 'GEN_CLOSED'
    

### Priming (기동) decorator

시작하기 전에 `next(mycoro)` 를 호출하지 않아도 되도록 아래의 `@coroutine` 을 널리 사용한다.
- `yield from` 은 아래의 `@coroutine` 과 함께 사용할 수 없다.
- `asyncio` 패키지의 `@coroutine`은 `yield from`과 함께 사용할 수 있도록 설계되었다. (18장)
- `coroutil` 패키지는 `PyPI`에 없다.

In [11]:
from functools import wraps

def coroutine(func):
    @wraps(func)
    def primer(*args, **kwargs):
        gen = func(*args, **kwargs)
        next(gen)
        return gen
    return primer

### `generator.throw(exc_type[, exc_value[, traceback]])` 와 `generator.close()`

In [12]:
from inspect import getgeneratorstate

class DemoException(Exception):
    pass

@coroutine
def demo_exc_handling():
    print('-> coroutine started')
    try:
        while True:
            try:
                x = yield
            except DemoException:
                print('*** DemoException handled. Continuing ...')
            else:
                print('-> coroutine received: {!r}'.format(x))
        raise RuntimeError('This line should never run')
    finally:
        print('-> coroutine ending')

In [13]:
exc_coro = demo_exc_handling()

-> coroutine started


In [14]:
exc_coro.send(11)

-> coroutine received: 11


In [15]:
exc_coro.close()

getgeneratorstate(exc_coro)

-> coroutine ending


'GEN_CLOSED'

In [16]:
exc_coro = demo_exc_handling()
exc_coro.send(11)
exc_coro.throw(DemoException)

getgeneratorstate(exc_coro)

-> coroutine started
-> coroutine received: 11
*** DemoException handled. Continuing ...


'GEN_SUSPENDED'

In [17]:
exc_coro.throw(ZeroDivisionError)

-> coroutine ending


ZeroDivisionError: 

In [18]:
getgeneratorstate(exc_coro)

'GEN_CLOSED'

`None` 이나 `Ellipsis` 구분 표시 등을 전달해서 종료할 수 있다.

### 코루틴에서 return 

값을 반환하려면 코루틴이 정상적으로 종료되어야 한다. PEP 380 참고. 

In [19]:
from collections import namedtuple

Result = namedtuple('Result', 'count average')

def averager():
    total = 0.0
    count = 0
    average = None
    while True:
        term = yield
        if term is None:
            break
        total += term
        count += 1
        average = total/count
    return Result(count, average)

In [20]:
coro_avg = averager()
next(coro_avg)
coro_avg.send(10)
coro_avg.send(30)
coro_avg.send(6.5)
try:
    coro_avg.send(None)
except StopIteration as exc:
    result = exc.value
result

Result(count=3, average=15.5)

### `yield from x`

- 모든 iterable(반복형)이 `x`에 사용될 수 있다. 
- 다른 언어의 await constructs와 유사하다. 
- "caller"가 "delegating generator"를 호출하면 `yield from`를 담고 있는 "delegating generator"가 "subgenerator"를 호출하고 subgenerator가 iterable(반복형)을 돌아 값을 생성후 delegating generator에 반환하며 subgenerator가 종료될 때까지 delegating generator는 실행을 중단한다. PEP 380 참고.

In [38]:
def chain(*iterables):
    for it in iterables:
        yield from it

s = 'ABC'
t = tuple(range(3))
list(chain(s, t))

['A', 'B', 'C', 0, 1, 2]

In [39]:
# the delegating generator
def grouper(results, key):
    while True:
        results[key] = yield from averager()

# the client code, a.k.a. the caller
def main(data):
    results = {}
    for key, values in data.items():
        group = grouper(results, key)
        next(group)
        for value in values:
            group.send(value)
        group.send(None) # important!
    # print(results)
    report(results)

# output report
def report(results):
    for key, result in sorted(results.items()):
        group, unit = key.split(';')
        print('{:2} {:5} averaging {:.2f}{}'.format(
              result.count, group, result.average, unit))
data = {
'girls;kg':
[40.9, 38.5, 44.3, 42.2, 45.2, 41.7, 44.5, 38.0, 40.6, 44.5],
'girls;m':
[1.6, 1.51, 1.4, 1.3, 1.41, 1.39, 1.33, 1.46, 1.45, 1.43],
'boys;kg':
[39.0, 40.8, 43.2, 40.8, 43.1, 38.6, 41.4, 40.6, 36.3],
'boys;m':
[1.38, 1.5, 1.32, 1.25, 1.37, 1.48, 1.25, 1.49, 1.46],
}

if __name__ == '__main__':
    main(data)

 9 boys  averaging 40.42kg
 9 boys  averaging 1.39m
10 girls averaging 42.04kg
10 girls averaging 1.43m


### 이산 이벤트 시뮬레이션 (Discrete Event Simulation)
- 고정된 값만큼 시계를 진행하는 것이 아니라 다음 이벤트의 시각으로 바로 진행한다.
- 코루틴은 DES를 작성하기 위한 추상화에 딱 맞는다.
- SimPy, asyncio, Twisted, Tornado : 단일 쓰레드 비동기화 

### 참고
- Beazley's PyCon tutorials
- Greedy algorithm with coroutines by James Powell
- great, detailed diagram by Paul Sokolovksy
- Ashish Gupta's Writing a Discrete Event Simulation: ten easy lessons
- PEP 380, PEP 492